In [1]:
import random
import math

# Initalisation Code Block

The below class initialises the Primer Design, and defined all the variables required to find the cost of each crtierion, where the forward and reverse primer should be chosen from and the parameters for the simulated annealing portion

In [2]:
class PrimerDesign(object):
    
    def __init__ (self, name):
        
        '''parameters for the length criterion'''
        self.max_length = 22
        self.min_length = 18
        self.penalty_length = 10
        
        '''parameters for the temperature difference criterion'''
        self.max_tdiff = 1
        self.min_tdiff = 0
        self.penalty_tdiff = 10
        
        '''parameters for the cg content criterion'''
        self.max_cg = 0.6
        self.min_cg = 0.4
        self.penalty_cg = 10
        
        '''parameters for the annealing temperature criterion'''
        self.max_temp = 63
        self.min_temp = 57
        self.penalty_temp = 10
        
        '''parameters for the run criterion'''
        self.run_threshold = 2
        self.penalty_runs = 10
        
        '''parameters for the repeat criterion'''
        self.repeat_threshold = 0
        self.penalty_repeats = 10
        
        '''parameters for the specificity criterion'''
        self.penalty_specificity = 10 
        
        '''locations where the forward primer should be chosen from'''
        self.fp_start = 0
        self.fp_end = 10
        
        '''locations where the reverse primer should be chosen from'''
        self.rp_start = 0
        self.rp_end = 10
        
        ''' parameters for the simulated annealing portion'''
        self.initial_temperature = 200
        self.stopping_temperature = 0.01
        self.drop_fraction = 0.999
        

# Setting the DNA Sequence

### Task 2

1. Take in an input ```dna_sequence```
2. Ensure that the input is a ```str```
3. Extract from the string only ```a,c,g,t``` and create a new string
4. Save the string in ```self.dna_sequence```

In [3]:
class PrimerDesign(PrimerDesign): 
    
    def set_dna_sequence(self, dna_sequence):
        #Condition1. the input arguments are of the correct type
        if type(dna_sequence) != str:
            print("Error: The DNA sequence needs to be a string")
            return None
        #Condition 2.the input arguments have reasonable values
        pass
    
        new_dna_sequence = ''
        
        for char in dna_sequence:
            if char in ("a", "c", "g", "t"):
                new_dna_sequence += char
        self.dna_sequence = new_dna_sequence

# Select a random forward or reverse primer

### Task 3

Condition 1:
1. Ensure ```sqtype``` is a ```str```
2. Ensure ```length``` is a ```int```

Condition 2:
1. Ensure ```sqtype``` is either ```'forward'``` or ```'reverse'```
2. Ensure ```length``` is greater than ```0``` but less than ```end_limit-start_limit```

Process:
1. Check if the forward or reverse sequence is required
2. Select a random starting point
3. Take characters, starting from the start point and ending ```length``` characters later
4. For both forward and reverse sequences the value returned is the same.
5. However in task 8 we are required to return the reverse compliment of the reverse sequence, which we will do in task 8

In [4]:
class PrimerDesign(PrimerDesign):
    
    def func_select_random(self, sqtype='forward', length = 20 ):
        
        start_limit = self.fp_start 
        end_limit = self.fp_end
        
        #Condition1. the input arguments are of the correct type
        if type(length) != int:
            print("Error: The length needs to be an integer")
            return None
        if type(sqtype) != str:
            print("Error: The sqtype needs to be a string")
            return None
        #Condition 2.the input arguments have reasonable values
        if sqtype != 'forward' and sqtype != 'reverse':
            print("Error: The sqtype needs to be either forward or reverse")
            return None
        if length <= 0:
            print("Error: The length needs to be a positive integer")
            return None
        if length >= end_limit-start_limit:
            print("Error: The length must be less than the difference between the end_limit and start_limit")
            return None
        
        '''the length has to be a positive number'''
        
        if(sqtype == 'forward'):
            starting_point = random.randint(start_limit, end_limit-length)
            rand_sequence = self.dna_sequence[starting_point:starting_point+length]
            return rand_sequence
            
        elif(sqtype == 'reverse'):
        
            starting_point = random.randint(start_limit, end_limit-length)
            rand_sequence = self.dna_sequence[starting_point:starting_point+length]
            return rand_sequence
        
        else: 
            return None
        



# For a given sequence, find length, cg fraction, and primer temperature

__length__ -> number of characters
__cg fraction__ -> ratio of ```c,g``` to ```a,t```
__primer temperature__ -> ideal annealing temperature

### Task 4

Condition 1:
1. Ensure the ```sq``` is a ```str```
Condition 2:
1. Ensure that ```sq``` only contains ```a,c,g,t```

Finding length:
1. Just return the ```len(sq)```

Finding sequence fraction
1. Obtain the length of sq using ```func_length```
2. Count the number of ```c,g```
3. Divide number of ```c,g``` by the ```len(sq)
4. Return the fraction

Finding temperature required:
1. Count the number of ```c,g```
2. Find the number of ```a,t``` by finding ```no of cg``` - ```len(sq)```
3. For every ```a,t``` annealing temperature ```+2```
4. For every ```c,g``` annealing temperature ```+4```
5. Return the temperature

In [5]:
class PrimerDesign(PrimerDesign):

    
    def func_length(self, sq):
        #Condition1. the input arguments are of the correct type
        if type(sq) != str:
            print("Error: sq needs to be a string")
            return None
        #Condition2. the input arguments must be of reasonable values
        for char in sq:
            if char != 'a' and char != 'g' and char != 'c' and char != 't':
                print("Error: sq can only contain a,c,g or t")
                return None
        
        return len(sq)
    
    def func_cg_fraction(self, sq):
        #Condition1. the input arguments are of the correct type
        if type(sq) != str:
            print("Error: sq needs to be a string")
            return None
        #Condition2. the input arguments must be of reasonable values
        for char in sq:
            if char != 'a' and char != 'g' and char != 'c' and char != 't':
                print("Error: sq can only contain a,c,g or t")
                return None
        
        sq_length = self.func_length(sq)
        
        number_of_cg = 0
        for char in sq:
            if char in ("c", "g"):
                number_of_cg += 1
        self.sq_cg_fraction = number_of_cg/sq_length
        
        return self.sq_cg_fraction
    
    def func_temperature(self,sq):
        #Condition1. the input arguments are of the correct type
        if type(sq) != str:
            print("Error: sq needs to be a string")
            return None
        #Condition2. the input arguments must be of reasonable values
        for char in sq:
            if char != 'a' and char != 'g' and char != 'c' and char != 't':
                print("Error: sq can only contain a,c,g or t")
                return None
        
        sq_length = self.func_length(sq)
        sq_cg_fraction = self.func_cg_fraction(sq)        
        
        no_of_cg = sq_cg_fraction*sq_length
        no_of_at = sq_length - no_of_cg
        temperature = 4*no_of_cg+2*no_of_at
        
        return temperature
    




# For a given sequence, find the number of runs

A run is considered if its length is greater than the run threshold, and each run is only considered once

### Task 4

Condition 1:
1. Ensure the ```sq``` is a ```str```
Condition 2:
1. Ensure that ```sq``` only contains ```a,c,g,t```

Process:
1. For every character in the sequence
2. If it is different from the previous character, end the run
3. Otherwise, add one to the number of same characters counted
4. If the number of same characters exceeds the run threshold
5. Add one to ```number_of_runs```
6. Do not overcount and add to number of runs once you have exceeded the run threshold

In [6]:
class PrimerDesign(PrimerDesign):

    def func_count_runs(self,sq):
        #Condition1. the input arguments are of the correct type
        if type(sq) != str:
            print("Error: sq needs to be a string")
            return None
        #Condition2. the input arguments must be of reasonable values
        for char in sq:
            if char != 'a' and char != 'g' and char != 'c' and char != 't':
                print("Error: sq can only contain a,c,g or t")
                return None
        
        number_of_runs = 0
        current_run_char = ''
        current_run_length = 0
        run_counted = False
        for char in sq:
            if char != current_run_char:
                current_run_char = char
                current_run_length = 0
                run_counted = False
            else:
                current_run_length += 1
                if current_run_length >= self.run_threshold and run_counted == False:
                    number_of_runs += 1
                    run_counted = True
        
        return number_of_runs

# For a given sequence, find the number of runs

A repeat is considered a repeat if a given sequence has the same 2 characters consecutively in the sequence

### Task 4

Condition 1:
1. Ensure the ```sq``` is a ```str```
Condition 2:
1. Ensure that ```sq``` only contains ```a,c,g,t```

Process:
1. Start the count from 0
2. Considering all the characters up to 3 characters from the end
3. If the character is the same as the character 2 characters after it eg. <b>A</b> C <b>A</b> C
4. And the next character is the same as the character 2 characters after it eg. A <b>C</b> A <b>C</b>
5. If the character is not the same as the next character eg. __A C__ A C (The first two are not the same)
6. Add one to the number of repeats

In [7]:
class PrimerDesign(PrimerDesign):
    def func_count_repeats(self,sq):
        #Condition1. the input arguments are of the correct type
        if type(sq) != str:
            print("Error: sq needs to be a string")
            return None
        #Condition2. the input arguments must be of reasonable values
        for char in sq:
            if char != 'a' and char != 'g' and char != 'c' and char != 't':
                print("Error: sq can only contain a,c,g or t")
                return None
        
        count=0
        for i in range(len(sq)-3):
            if sq[i] == sq[i+2] and sq[i+1] == sq[i+3]:
                if sq[i] != sq[i+1]:
                    count += 1
        return count

# For a given sequence, find the cost of the different variables
### List of costs required:
__cost length__<br>
__cost temperature__<br>
__cost cgcontent__<br>
__cost temperature different__<br>
__cost specifity__<br>
__cost runs__<br>
__cost repeats__<br>


### Task 5

Cost Length:
1. If the length exceeds the max or min
2. Find the difference between the value it has exceeded
3. Multiply it by the penalty

Cost temperature
1. If the temperature exceed the max or min
2. Find the difference between the max or min
3. Multiply it by the penalty

Cost CG content
1. If the CG content exceeds the max or min
2. Find the difference between the max or min
3. Multiply it by the penalty

Cost Temperature Difference
1. If the temperature difference exceeds the max
2. Find the difference between the max
3. Multiply it by the penalty

Cost Specifity
1. Find the number of similar sequences in the DNA
2. Take the ```total number of sequences``` - ```1```
3. Multiply it by the penalty

Cost runs
1. Find the number of runs
2. Multiply it by the penalty

Cost Repeats
1. Find the number of repeats
2. Multiply it by the penalty

In [8]:
class PrimerDesign(PrimerDesign):
    
    def cost_length(self, sq):
        '''This is given to you as an example '''
        sq_len = self.func_length(sq)
        if(sq_len > self.max_length):
            return (sq_len - self.max_length)*self.penalty_length
        elif(sq_len > self.min_length):
            return 0
        else:
            return (self.min_length - sq_len)*self.penalty_length 
    
    def cost_temperature(self, sq):
        sq_temp = self.func_temperature(sq)
        if(sq_temp>self.max_temp):
            return (sq_temp-self.max_temp)*self.penalty_temp
        elif(sq_temp>self.min_temp):
            return 0
        else:
            return (self.min_temp-sq_temp)*self.penalty_temp
        
    def cost_cgcontent(self,sq):
        sq_cgcontent = self.func_cg_fraction(sq)
        if(sq_cgcontent>0.6):
            return (sq_cgcontent-0.6)*self.penalty_cg
        elif(sq_cgcontent>0.4):
            return 0
        else:
            return (0.4-sq_cgcontent)*self.penalty_cg
        
    def cost_temperature_difference(self, fp, rp):
        fp_temp = self.func_temperature(fp)
        rp_temp = self.func_temperature(rp)
        temp_diff = abs(fp_temp-rp_temp)
        if temp_diff>self.max_tdiff:
            return (temp_diff-self.max_tdiff)*self.penalty_tdiff
        else:
            return 0
    
    def cost_specificity(self, sq):
        number_of_instances = self.dna_sequence.count(sq)
        return (number_of_instances-1)*self.penalty_specificity
    
    def cost_runs(self, sq):
        number_of_runs = self.func_count_runs(sq)
        return number_of_runs*self.penalty_runs
        
    def cost_repeats(self,sq):
        number_of_repeats = self.func_count_repeats(sq)
        return number_of_repeats*self.penalty_repeats

# For a given sequence, find the total cost

### Task 6

Process:
1. Find the cost due to:<br>
repeats,<br>
runs,<br>
specificity,<br>
cgcontent,<br>
temperature,<br>
length

2. Add all the values together
3. return the total cost

In [9]:
class PrimerDesign(PrimerDesign):
    
    def cost_objective_function(self, fp, rp):
        '''complete the calculation of the cost'''
        cost = 0
        costs_list = ['cost_repeats',
                     'cost_runs',
                     'cost_specificity',
                     'cost_cgcontent',
                     'cost_temperature',
                     'cost_length']
        for function in costs_list:
            cost += getattr(self,function)(fp)
            cost += getattr(self,function)(rp)
        cost += self.cost_temperature_difference(fp,rp)
        return cost

      

# Print the costs for the forward primer and the reverse primer nicely

### Task 7

In [65]:


class PrimerDesign(PrimerDesign):
    
    def create_information_row(self,*args):
        #Criterion, Actual, Minimum, Maximum, Cost Function Score
        #print(kwargs)
        numbers = False
        for item in args:
            if type(item) == float or type(item) == int:
                numbers = True
        if numbers == False:
            text = '{:<22}{:>10}{:>10}{:>10}{:>21}'.format(*args)
        else:
            text = '{:<22}{:>10.3f}{:>10.3f}{:>10.3f}{:>21.3f}'.format(*args)
        if args[-1] == 'Cost Function Score':
            text += '{:>18}'.format('Objective Met?')
        elif args[-1] == 0:
            text += '{:>18}'.format('YES')
        else:
            text += '{:>18}'.format('NO')
            
        return text
    
    def right_align(self,name,cost):
        if type(cost) == str:
            cost = '{}'.format(cost) 
        else:
            cost = '{:.3f}'.format(cost)
        text = ''
        text += name
        spaces_required = 50-len(name)-len(str(cost))
        for i in range(spaces_required):
            text += ' '
        text += '{}'.format(cost) 
        return text
    
    def cost_objective_function_info(self, fp, rp):
        bases = {"c": "g", "g": "c", "a": "t", "t": "a"}
        reverse = rp[::-1]
        complement = ""
        for base in reverse:
            complement += bases[base]
        
        
        text = ''
        
        for i,x in enumerate([fp,rp]):

            if i == 0:
                text += '===Forward Primer=== '+ x + '\n'
            elif i == 1:
                text += '===Reverse Primer=== '+ complement + '\n'
            text += Primer.create_information_row('Criterion','Actual','Min','Max','Cost Function Score') + '\n'
            text += Primer.create_information_row('length',\
                                                  self.func_length(x),\
                                                  self.min_length,\
                                                  self.max_length,\
                                                  self.cost_length(x)) + '\n'
            text += Primer.create_information_row('annealing temperature',\
                                                  self.func_temperature(x),\
                                                  self.min_temp,\
                                                  self.max_temp,\
                                                  self.cost_temperature(x)) + '\n'
            text += Primer.create_information_row('%cg_content',\
                                                  self.func_cg_fraction(x),\
                                                  self.min_cg,\
                                                  self.max_cg,\
                                                  self.cost_cgcontent(x)) + '\n'
            text += Primer.create_information_row('specificity',\
                                                  float(self.dna_sequence.count(x)),\
                                                  1,\
                                                  1,\
                                                  self.cost_specificity(x)) + '\n'
            text += Primer.create_information_row('runs',\
                                                  self.func_count_runs(x),\
                                                  0,\
                                                  self.run_threshold,\
                                                  self.cost_runs(x)) + '\n'
            text += Primer.create_information_row('repeats',\
                                                  self.func_count_repeats(x),\
                                                  0,\
                                                  self.repeat_threshold,\
                                                  self.cost_repeats(x)) + '\n'
            text += '\n'
    
        fp_temp = self.func_temperature(fp)
        rp_temp = self.func_temperature(rp)
        temp_diff = abs(fp_temp-rp_temp)
        
        text += Primer.create_information_row('Temperature Difference',\
                                              temp_diff,\
                                              0,\
                                              self.max_tdiff,\
                                              self.cost_temperature_difference(fp,rp)) + '\n'      

        return text

Primer = PrimerDesign("dna")
Primer.set_dna_sequence(dna_sequence)
Primer.fp_start = 670
Primer.fp_end = 1165
Primer.rp_start = 1181
Primer.rp_end = 1680
fp_current,rp_current = Primer.func_simulated_annealing()
m = Primer.cost_objective_function_info(fp_current, rp_current)
print(m)


===Forward Primer=== ccaacaggccttgatgaggt
Criterion                 Actual       Min       Max  Cost Function Score    Objective Met?
length                    20.000    18.000    22.000                0.000               YES
annealing temperature     62.000    57.000    63.000                0.000               YES
%cg_content                0.550     0.400     0.600                0.000               YES
specificity                1.000     1.000     1.000                0.000               YES
runs                       0.000     0.000     2.000                0.000               YES
repeats                    0.000     0.000     0.000                0.000               YES

===Reverse Primer=== agcctgacatgatggcatcg
Criterion                 Actual       Min       Max  Cost Function Score    Objective Met?
length                    20.000    18.000    22.000                0.000               YES
annealing temperature     62.000    57.000    63.000                0.000              

### Task 10

In [16]:
import numpy as np

class PrimerDesign(PrimerDesign): 
    def get_primer_neighbor(self, sq):
        x = self.dna_sequence
        original_position = x.find(sq)
        new_position = original_position + random.randint(-22, 22) 
        if new_position in range(self.fp_start, self.fp_end+1) or new_position in range(self.rp_start, self.rp_end+1):
            new_length = random.randint(self.min_length, self.max_length)
            new_sq = ''
            for i in range(new_position, new_position + new_length):
                new_sq += self.dna_sequence[i]
            return new_sq
        else:
            return self.get_primer_neighbor(sq)

    
    def func_simulated_annealing(self):
        
        temperature = self.initial_temperature
        stopping_temperature = self.stopping_temperature
        drop = self.drop_fraction
        fp_current = self.func_select_random('forward', length = 20)
        rp_current = self.func_select_random('reverse', length = 20)
        current_cost = self.cost_objective_function(fp_current, rp_current)
        
        while temperature > stopping_temperature:
            fp_new = self.get_primer_neighbor(fp_current)
            rp_new = self.get_primer_neighbor(rp_current)
            new_cost = self.cost_objective_function(fp_new, rp_new)
            if new_cost < current_cost:
                fp_current = fp_new
                rp_current = rp_new
                current_cost = new_cost
            else:
                delta = new_cost - current_cost
                acceptance_probability = np.exp(-delta/temperature)
                num = random.random()
                
                if acceptance_probability > num:
                    fp_current = fp_new
                    rp_current = rp_new
                    current_cost = new_cost
                    
            temperature = temperature * drop
        return fp_current, rp_current       

# Testing stuff

#Primer = PrimerDesign("gay")
#Primer.set_dna_sequence(dna_sequence)
#print(Primer.dna_sequence)  

#fp = Primer.func_select_random(sqtype='forward', length = 5)
#rp = Primer.func_select_random(sqtype='reverse', length = 5)
#print(fp)
#print(rp)

#fp_current,rp_current = Primer.func_simulated_annealing()



===Forward Primer=== tgaagaaggattcttatgctcg
Criterion                      Cost Function Score
length                                       0.000
annealing temperature                        0.000
%cg_content                                  0.000
specificity                                  0.000
runs                                         0.000
repeats                                      0.000

===Reverse Primer=== catgatggcatcgttggatga
Criterion                      Cost Function Score
length                                       0.000
annealing temperature                        0.000
%cg_content                                  0.000
specificity                                  0.000
runs                                         0.000
repeats                                      0.000

Temperature Difference                       0.000


### Store the DNA sequence given to you in the variable below 

In [12]:
dna_sequence = '''gattggctgg cgcggattcc agctgctttc caagtcagcg gcgcctagtg agagtcaggg
gggcccggcc cgcgccctcc ccgcccagcc gcctccccgt cgacgcccag ggctggggcg
agccaggctg cctttcgaac ttggggggct tctcctcttg tctcccactg gtgctctggc
tgtgaatcca tccaggggcc aggatgacaa tccgacacca aggccagcag tacaggccga
ggatggcatt tctccaaaag attgaagcgc tcgtgaagga catgcagaac ccagagacag
gggtccgaat gcagaaccag agggtcctgg tcaccagcgt tcctcatgcc atgacaggaa
gtgatgttct gcaatggatc gtccagcggc tttggatctc cagtctggag gcacagaact
tgggcaactt tattgtcagg tatggctaca tttaccccct gcaagacccc aagaatctca
481 ttctcaagcc tgatggcagc ctctacagat ttcagacacc gtatttctgg cccacccagc
541 agtggccagc tgaagatacc gattacgcca tctatctggc caagcgaaat atcaaaaaga
601 aagggatttt ggaagaatat gaaaaggaaa attacaattt cttgaaccaa aaaatgaact
661 ataagtggga ctttgtcatt atgcaggcca aagagcagta cagggctgga aaggagagga
721 acaaagcaga cagatatgcc ctggactgcc aggagaaggc atactggctg gtgcaccgat
781 gccctcctgg aatggacaat gtgctggact acggcctgga ccgagtgacc aatccgaatg
841 aagtcaaggt aaaccagaaa caaacagtcg ttgctgtcaa aaaagagatc atgtattacc
901 aacaggcctt gatgaggtcc acagtgaagt cttctgtgtc cctgggaggg attgtgaaat
961 acagtgagca gttctcatcc aacgatgcca tcatgtcagg ctgcctcccc agcaacccct
1021 ggatcaccga tgacacccag ttctgggact taaatgccaa attggtggaa atcccaacca
1081 agatgcgagt ggaacgatgg gccttcaact tcagcgaatt gatccgagac cccaaaggtc
1141 gacagagctt ccagtacttc ctcaagaaag aattcagtgg agagaatctg ggattctggg
1201 aagcctgcga ggatctgaag tatggagatc agtccaaagt caaggagaaa gcagaggaga
1261 tttacaagct gttcctggcc atttgggcga ggcgctggat caacatagat ggcaaaacca
1321 tggacatcac agtgaagggg ctgaagcacc cccaccgcta tgtgctggac gccgcacaaa
1381 cccacattta catgctcatg aagaaggatt cttatgctcg ctatgcccaa tctccgatct
1441 ataaggacat gctggccaaa gctattgaac ctcaggaaac caccaagaaa agctccaccc
1501 tcccttttat gcggcgtcac ctgcgctcca gcccaagccc tgtcatcctg agacagctgg
1561 aagaggaagc caaggcccga gaagcagcca acactgtgga catcacccag ccgggccagc
1621 acatggctcc cagcccccat ctgaccgtgt acaccgggac ctgcatgccc ccgtctcctt
1681 ctagcccctt ctcctcctcc tgccgctccc ccaggaagcc tttcgcctca cccagccgct
1741 tcatccggcg acccagcacc accatctgcc cctcacccat cagagtggcc ttggagagct
1801 catcgggctt ggagcagaaa ggggagtgca gcgggtccat ggccccccgt gggccctctg
1861 tcaccgagag cagcgaggcc tccctcgaca cctcctggcc tcgcagccgg cccagggccc
1921 ctcctaaggc ccgcatggct ctgtccttca gcaggtttct gagacgaggc tgtctggcct
1981 cacctgtctt tgccaggctc tcacccaagt gccctgctgt gtcccacggg agggtgcagc
2041 ccctggggga cgtgggccag cagctgccac gattgaaatc caagagagta gcaaactttt
2101 tccagatcaa aatggatgtg cccacgggga gcgggacctg cttgatggac tcggaggatg
2161 ctggaacagg agagtcgggt gaccgggcca cagaaaagga ggtcatctgc ccctgggaga
2221 gcctgtaagg aaagaggcag gctgagctgg gggctctgga ccaggaagat gctctgacag
2281 atgccatggt atgggccaca ggacacactt gctcgagaac caaagtgcat ttgggtgaca
2341 tttgaagatt ggggagacaa gatggggtag attgtggcaa agaatgctct ggctggttac
2401 caggggccaa ctccttctcc tcttcctgac cctccctccc ctgggcagaa gaaacgcatg
2461 tggaccagaa gactttccct gctgccttaa aaccaataaa aggttaactt taagtttctt
2521 ggaaaaaaaa aaaaaaaaaa
'''

### Instantiate your class and read in the DNA sequence

In [13]:
Primer = PrimerDesign("dna")
Primer.set_dna_sequence(dna_sequence)

### If you need to adjust any parameter from their default values in the init method, do it here

In [14]:
Primer.fp_start = 670
Primer.fp_end = 1165
Primer.rp_start = 1181
Primer.rp_end = 1680

### Show the outcome of your testing and the functions in the subsequent cells 

In [15]:
fp_current,rp_current = Primer.func_simulated_annealing()
m = Primer.cost_objective_function_info(fp_current, rp_current)
print(m)

===Forward Primer=== cgagtgaccaatccgaatga
Criterion                      Cost Function Score
length                                       0.000
annealing temperature                        0.000
%cg_content                                  0.000
specificity                                  0.000
runs                                         0.000
repeats                                      0.000

===Reverse Primer=== ggaggcagcctgacatgat
Criterion                      Cost Function Score
length                                       0.000
annealing temperature                        0.000
%cg_content                                  0.000
specificity                                  0.000
runs                                         0.000
repeats                                      0.000

Temperature Difference                       0.000
